# Brex Take Home Assignment Instructions

## Executive Summary

Thank you for applying to Brex and congratulations on making it to the take-home assignment. Using this assignment, we will evaluate your ability to both comprehend a dataset and draw insights in a timely manner. We will also evaluate your attention to detail, and ability to find errors in the dataset.

Please answer the following questions in the cells below. Add as many cells as needed, as well as any diagrams or charts you feel should be included as part of the answer. Don't feel obligated to use Python as the designated language, as you can use any language (R, SQL, C++, Java) or platform/notebook (Rstudio, PostgreSQL, Databricks, etc). Just copy the markdown and fill in your code, as we need to review both your code and the answer.

Please submit your answers to questions 1-4 as the file submission.csv in the same format as `submission_example.csv` and `financial_balances_submission.csv`. This is critical in your grade as files with wrong name (e.g. "submission_test.csv") or wrong format (e.g. "1.a","aaa") will result in point reduction. Also, please submit your `.ipynb` notebook or any of the accompanying code (Java, R, SQL, etc) you wrote to answer the questions. In short, you MUST submit the following:

  * submission.csv
  * any notebook equivalent with your code

Make sure that you go through the questions in ascending order (1, 2, 3, 4), as the subsequent questions build on top of the previous questions. The maximum time limit is `2 hours and 30 minutes`. Please adhere to the time limit and refer to the recommended time usage on each question to gauge the effort required to answer.

## Background
The main goal for this case is for us to understand your ability to understand the data provided and draw critical insights from the dataset. There are four tables for you to work with: 
1. Customer Accounts (id, name): a list of our customers with their names redacted and a unique identifier
2. Financial Accounts (id, customer_account_id, name): a list of bank accounts belonging to our customers
    * Customer_account_id: The ID of the customer that owns this bank account from Customer Accounts (table 1)
    * Id, name: Unique financial account identifier and redacted name of the account
3. Financial Balances (id, account_id, amount, accrual_date): a reading of the beginning-of-day (BoD) balance for each financial account
    * Account_id: the ID of the financial account (table 2) that the balance belongs to
    * Amount and accrual_date: The balance (in dollars) and date this reading was taken
4. Financial Transactions (id, account_id, amount, accrual_date): list of transactions incoming and outgoing for each financial account
    * Account_id: the ID of the financial account (table 2) that the transaction belongs to
    * Amount and accrual_date: the amount (in dollars) of each transaction and the date when the transaction occurred. Negative transactions are money leaving the bank account and positive transactions are money coming in.

Using these accounts, please answer the following questions in the sample jupyter notebook provided. Scores for questions 1-4 will be graded based on your submitted csv. Please read through the submission file format in `submission_example.csv` & `financial_balances_submission.csv` and include the finalized submissions in  `submission.csv` & `financial_balances_submission.csv`.

### Feel free to use any many cells as you need to answer the question and please keep in mind that someone will be reading your code, so be thoughtful to use comments and organize your code.

## Notice that scores for questions 1-4 will be graded based on your submitted csv files. Please read through the submission file format and include the finalized submissions.

## Warm up
Recommended time usage: ~10 minutes

Perform a quick Exploratory Dataset Analysis (EDA) on the dataset provided. Draw some insights on the raw data. ( 0 points )


In [86]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import os

In [4]:
os.getcwd()

'/home/jupyter/Brex Data Challenge'

In [46]:
# list all of files in directory
files = os.listdir()

[f for f in files]

['financial_accounts.csv',
 'customer_accounts.csv',
 'financial_balances_submission.csv',
 'submission_example.csv',
 'Brex Take Home Instructions.ipynb',
 'Brex Take Home Instructions.pdf',
 'financial_transactions.csv',
 'financial_balances.csv',
 'submission.csv',
 '.ipynb_checkpoints']

In [6]:
# import data 
customer_accounts = pd.read_csv('./customer_accounts.csv')
financial_accounts = pd.read_csv('./financial_accounts.csv')
financial_transactions = pd.read_csv('./financial_transactions.csv')
financial_balances = pd.read_csv('./financial_balances.csv')

In [7]:
print(customer_accounts.head(),' \n', customer_accounts.shape)
print(financial_accounts.head(),' \n', financial_accounts.shape)
print(financial_transactions.head(),' \n', financial_transactions.shape)
print(financial_balances.head(),' \n', financial_balances.shape)

    id         name
0  a_0  a_0_company
1  a_1  a_1_company
2  a_2  a_2_company
3  a_3  a_3_company
4  a_4  a_4_company  
 (150, 2)
      id customer_account_id               name
0  a_0_0                 a_0  a_0_0_fin_account
1  a_0_1                 a_0  a_0_1_fin_account
2  a_0_2                 a_0  a_0_2_fin_account
3  a_0_3                 a_0  a_0_3_fin_account
4  a_0_4                 a_0  a_0_4_fin_account  
 (590, 3)
  account_id accrual_date  amount       id
0      a_0_0   2019-12-24  -248.0  a_0_0_0
1      a_0_0   2019-12-25  -859.0  a_0_0_1
2      a_0_0   2019-12-26  -776.0  a_0_0_2
3      a_0_0   2019-12-27  -605.0  a_0_0_3
4      a_0_0   2019-12-28   -33.0  a_0_0_4  
 (222723, 4)
            id account_id accrual_date     amount
0  a_0_0_bal_0      a_0_0   2019-12-24  9950997.0
1  a_0_0_bal_1      a_0_0   2019-12-25  9950749.0
2  a_0_0_bal_2      a_0_0   2019-12-26  9949890.0
3  a_0_0_bal_3      a_0_0   2019-12-27  9949114.0
4  a_0_0_bal_4      a_0_0   2019-12-28  99485

In [47]:
# check missing value
# As we can see only one column contains missing value which is amount in financial_balances.csv
# We will be handling missing values later and coming up solution to fill in these missing values
print(customer_accounts.isnull().sum())
print(financial_accounts.isnull().sum())
print(financial_transactions.isnull().sum())
print(financial_balances.isnull().sum())

id      0
name    0
dtype: int64
id                     0
customer_account_id    0
name                   0
dtype: int64
account_id      0
accrual_date    0
amount          0
id              0
dtype: int64
id               0
account_id       0
accrual_date     0
amount          90
dtype: int64


In [8]:
# combine the data
# Step1: combine customer_accounts & financial_accounts table
customer_financial_accounts = pd.merge(customer_accounts.rename(columns={'id':'customer_account_id','name':'customer_name'}),
                                       financial_accounts.rename(columns={'id':'account_id','name':'account_name'}),
                                       how='left', 
                                       on='customer_account_id')

# Step2: combine customer_financial_accounts & financial_transactions
customer_financial_accounts_transactions = pd.merge(financial_transactions.rename(columns={'id':'transaction_id', 'amount':'transaction_amount'}), 
                                                    customer_financial_accounts, 
                                                    how='left',
                                                    on='account_id')
# Step3: Join balance and transaction table 
df_transactions_balances = pd.merge(customer_financial_accounts_transactions,
                                    financial_balances.rename(columns={'id':'balance_id', 'amount':'balance_amount'}),
                                    how='left',
                                    on=['account_id', 'accrual_date' ])

In [9]:
df_transactions_balances.head()

,account_id,accrual_date,transaction_amount,transaction_id,customer_account_id,customer_name,account_name,balance_id,balance_amount
0,a_0_0,2019-12-24,-248.0,a_0_0_0,a_0,a_0_company,a_0_0_fin_account,a_0_0_bal_0,9950997.0
1,a_0_0,2019-12-25,-859.0,a_0_0_1,a_0,a_0_company,a_0_0_fin_account,a_0_0_bal_1,9950749.0
2,a_0_0,2019-12-26,-776.0,a_0_0_2,a_0,a_0_company,a_0_0_fin_account,a_0_0_bal_2,9949890.0
3,a_0_0,2019-12-27,-605.0,a_0_0_3,a_0,a_0_company,a_0_0_fin_account,a_0_0_bal_3,9949114.0
4,a_0_0,2019-12-28,-33.0,a_0_0_4,a_0,a_0_company,a_0_0_fin_account,a_0_0_bal_4,9948509.0


In [57]:
# check if all the balance_id is coming through after blending the data
df_transactions_balances.loc[df_transactions_balances.balance_id.isnull()].head(5)

,account_id,accrual_date,transaction_amount,transaction_id,customer_account_id,customer_name,account_name,balance_id,balance_amount
2909,a_1_0,2020-02-28,-691.0,a_1_0_0,a_1,a_1_company,a_1_0_fin_account,NaN,NaN
2910,a_1_0,2020-02-29,-820.0,a_1_0_1,a_1,a_1_company,a_1_0_fin_account,NaN,NaN
2911,a_1_0,2020-03-01,-624.0,a_1_0_2,a_1,a_1_company,a_1_0_fin_account,NaN,NaN
2912,a_1_0,2020-03-02,-972.0,a_1_0_3,a_1,a_1_company,a_1_0_fin_account,NaN,NaN
2913,a_1_0,2020-03-03,-510.0,a_1_0_4,a_1,a_1_company,a_1_0_fin_account,NaN,NaN


In [60]:
# there seems to be missing daily balance for some account_id on a certain dates 
# which led to balance_id and balance_amount to be NULLs
# for instance account_id: a_1_0 on accrual_date of '2020-02-28'
# we will deal with it in a moment
financial_balances.loc[(financial_balances.account_id == 'a_1_0') & (financial_balances.accrual_date == '2020-02-28')]

,id,account_id,accrual_date,amount


## Questions

For the following questions:
- `Customer A` = "a_53"
- `Customer A Account 1` = "a_53_1" 
- `Customer A Balance Date` = "2020-07-04
- `Customer B` = "a_32" 
- `Customer B Balance Date` = "2020-08-12"

In [61]:
# recommended to run
customer_a = 'a_53'
customer_a_account_1 = 'a_53_1'
customer_a_balance_date = '2020-07-04'
customer_b = 'a_32'
customer_b_balance_date= '2020-08-12'

### Question 1.
Recommended time usage: ~10 minutes

(a) What is the average historic daily balance (beginning-of-day) for `Customer A Account 1`? Round to the nearest integer. ( 1 Point )

(b) What is `Customer A`'s global balance (sum of all beginning-of-day balances across all financial accounts associated with `Customer A`) on `Customer A Balance Date`? Round to the nearest integer. ( 1 Point )


In [65]:
# use customer_a and/or customer_a_account_1
# (a) Average historical daily balance for customer A account 1
avg_balance = df_transactions_balances.loc[(df_transactions_balances.customer_account_id == customer_a) & 
                             (df_transactions_balances.account_id == customer_a_account_1),
                             'balance_amount'].mean().round()

print('The avergae historic daily balance for Customer A Account 1 is {}.'.format(avg_balance))

The avergae historic daily balance for Customer A Account 1 is 10008868.0.


In [64]:
# double check the result with the original table
financial_balances.loc[financial_balances.account_id == customer_a_account_1, 'amount'].mean()

10008868.293103449

In [15]:
# (b) Customer A's global balnce
sum_of_global_balance = df_transactions_balances.loc[(df_transactions_balances.customer_account_id == customer_a) &
                             (df_transactions_balances.accrual_date == customer_a_balance_date),
                             'balance_amount'].unique().sum()
print('Customer A global balance on {0} is {1}'.format(customer_a_balance_date, sum_of_global_balance))

Customer A global balance on 2020-07-04 is 39806277.0


In [16]:
df_transactions_balances.loc[(df_transactions_balances.customer_account_id == customer_a) &
                             (df_transactions_balances.accrual_date == customer_a_balance_date)]

,account_id,accrual_date,transaction_amount,transaction_id,customer_account_id,customer_name,account_name,balance_id,balance_amount
82158,a_53_0,2020-07-04,-757.0,a_53_0_864,a_53,a_53_company,a_53_0_fin_account,a_53_0_bal_57478,9583746.0
82159,a_53_0,2020-07-04,-498.0,a_53_0_865,a_53,a_53_company,a_53_0_fin_account,a_53_0_bal_57478,9583746.0
82160,a_53_0,2020-07-04,-640.0,a_53_0_866,a_53,a_53_company,a_53_0_fin_account,a_53_0_bal_57478,9583746.0
82161,a_53_0,2020-07-04,-582.0,a_53_0_867,a_53,a_53_company,a_53_0_fin_account,a_53_0_bal_57478,9583746.0
82162,a_53_0,2020-07-04,-372.0,a_53_0_868,a_53,a_53_company,a_53_0_fin_account,a_53_0_bal_57478,9583746.0
82163,a_53_0,2020-07-04,-1043.0,a_53_0_869,a_53,a_53_company,a_53_0_fin_account,a_53_0_bal_57478,9583746.0
82164,a_53_0,2020-07-04,-193.0,a_53_0_870,a_53,a_53_company,a_53_0_fin_account,a_53_0_bal_57478,9583746.0
82165,a_53_0,2020-07-04,-304.0,a_53_0_871,a_53,a_53_company,a_53_0_fin_account,a_53_0_bal_57478,9583746.0
82687,a_53_1,2020-07-04,-731.0,a_53_1_56,a_53,a_53_company,a_53_1_fin_account,a_53_1_bal_57594,10009497.0
82857,a_53_2,2020-07-04,-1177.0,a_53_2_110,a_53,a_53_company,a_53_2_fin_account,a_53_2_bal_57764,10022075.0


In [17]:
df_transactions_balances.head()

,account_id,accrual_date,transaction_amount,transaction_id,customer_account_id,customer_name,account_name,balance_id,balance_amount
0,a_0_0,2019-12-24,-248.0,a_0_0_0,a_0,a_0_company,a_0_0_fin_account,a_0_0_bal_0,9950997.0
1,a_0_0,2019-12-25,-859.0,a_0_0_1,a_0,a_0_company,a_0_0_fin_account,a_0_0_bal_1,9950749.0
2,a_0_0,2019-12-26,-776.0,a_0_0_2,a_0,a_0_company,a_0_0_fin_account,a_0_0_bal_2,9949890.0
3,a_0_0,2019-12-27,-605.0,a_0_0_3,a_0,a_0_company,a_0_0_fin_account,a_0_0_bal_3,9949114.0
4,a_0_0,2019-12-28,-33.0,a_0_0_4,a_0,a_0_company,a_0_0_fin_account,a_0_0_bal_4,9948509.0


In [70]:
# validate the result by checking the original table
# how many account does Customer A have?
customer_a_acct = financial_accounts.loc[financial_accounts.customer_account_id == customer_a, 'id'].unique().tolist()

In [71]:
customer_a_acct

['a_53_0', 'a_53_1', 'a_53_2', 'a_53_3']

In [77]:
# the result from calculating from the original table is consistent with what we saw earlier
financial_balances.loc[(financial_balances.account_id.isin(customer_a_acct)) & (financial_balances.accrual_date == customer_a_balance_date), 'amount'].sum()

39806277.0

### Question 2.

Recommended time usage: ~10 minutes

`financial_balances.csv` shows you the beginning-of-day (BoD) balance, but transactions happened throughout the day. Calculate the end-of-day (EoD) global balance for `Customer A` on `Customer A Balance Date`. ( 1 point )

In [18]:
# use customer_a and/or customer_a_account_1 and/or customer_a_balance_date
customerA = df_transactions_balances.loc[(df_transactions_balances.customer_account_id == customer_a) &
                                         (df_transactions_balances.accrual_date == customer_a_balance_date)]
customerA_agg = customerA.groupby(by=['account_id', 'accrual_date','balance_amount'], as_index=False).agg(total_transaction_amount = ('transaction_amount','sum'))
customerA_agg

,account_id,accrual_date,balance_amount,total_transaction_amount
0,a_53_0,2020-07-04,9583746.0,-4389.0
1,a_53_1,2020-07-04,10009497.0,-731.0
2,a_53_2,2020-07-04,10022075.0,-1177.0
3,a_53_3,2020-07-04,10190959.0,-185.0


In [19]:
customerA_agg['balance_amount_EoD'] = customerA_agg['balance_amount'] + customerA_agg['total_transaction_amount']
customerA_agg

,account_id,accrual_date,balance_amount,total_transaction_amount,balance_amount_EoD
0,a_53_0,2020-07-04,9583746.0,-4389.0,9579357.0
1,a_53_1,2020-07-04,10009497.0,-731.0,10008766.0
2,a_53_2,2020-07-04,10022075.0,-1177.0,10020898.0
3,a_53_3,2020-07-04,10190959.0,-185.0,10190774.0


In [20]:
# validate the result through using the beginning-of-day (BoD) balance in financial_balances.csv
# it is apparent that the BoD balance on '2020-07-05' is consistent with the EoD balance that we calculated on '2020-07-04'
df_transactions_balances.loc[(df_transactions_balances.customer_account_id == customer_a) &
                            (df_transactions_balances.accrual_date == '2020-07-05'),
                            ['account_id','accrual_date','customer_account_id','balance_amount']].drop_duplicates()

,account_id,accrual_date,customer_account_id,balance_amount
82166,a_53_0,2020-07-05,a_53,9579357.0
82688,a_53_1,2020-07-05,a_53,10008766.0
82858,a_53_2,2020-07-05,a_53,10020898.0
82977,a_53_3,2020-07-05,a_53,10190774.0


In [21]:
customerA_globel_balance = customerA_agg['balance_amount_EoD'].sum()
print('the end-of-day (EoD) global balance for Customer A on Customer A Balance Date is {}'.format(customerA_globel_balance))

the end-of-day (EoD) global balance for Customer A on Customer A Balance Date is 39799795.0


In [80]:
# validating the results by calculate the difference between sum of transactions and the sum of balances across accounts
transactional_amount = financial_transactions.loc[(financial_transactions.account_id.isin(customer_a_acct)) & (financial_transactions.accrual_date == customer_a_balance_date), 'amount'].sum()
balance_amount = financial_balances.loc[(financial_balances.account_id.isin(customer_a_acct)) & (financial_balances.accrual_date == customer_a_balance_date), 'amount'].sum()
balance_amount + transactional_amount 

39799795.0

### Question 3.
Recommended time usage: ~20 minutes

(a) Among the `customer_account_id`s without any missing or null/NaN beginning-of-day balance for any date in between their first and their last `accrual_date`s, which `customer_account_id` has the largest variance of the sums of daily transactions amounts across all accounts? ( 1 points )

(b) Compute the `end-of-day` balances for all `financial_accounts`. Fill in as many missing dates as possible. The final dataset should include daily `end-of-day` balances for all `financial_accounts` as a csv in a similar format as `financial_balances.csv` (include just those columns: [`account_id, accrual_date, amount`]) and named `financial_balances_submission.csv`. ( 10 points )

In [81]:
# step 1: generate a list of customers with at least one missing/null/NaN balance for any date between 
# checking missing value 
# we can see that there are two missing data condition - either missing balance_id or missing balance_amount
print(df_transactions_balances.isnull().sum())
customer_with_null = df_transactions_balances.loc[df_transactions_balances.balance_amount.isnull(), 'customer_account_id'].unique().tolist()
customer_unique = df_transactions_balances.loc[:,'customer_account_id'].nunique()
print('{0} out of {1} total customers who have at least one missing/null/NaN BoD balance'.format(len(customer_with_null), customer_unique))

account_id                 0
accrual_date               0
transaction_amount         0
transaction_id             0
customer_account_id        0
customer_name              0
account_name               0
balance_id             14462
balance_amount         14686
dtype: int64
85 out of 150 total customers who have at least one missing/null/NaN BoD balance


In [119]:
# step 2: filter out those customer IDs with missing value of BoD balance 
df_customer_wt_missing = df_transactions_balances[~df_transactions_balances.customer_account_id.isin(customer_with_null)]
# step 3: sum up the transaction amount at daily basis
df_customer_wt_missing_agg = df_customer_wt_missing.groupby(by=['customer_account_id','account_id', 'accrual_date','balance_amount'], as_index=False).agg(total_transaction_amount = ('transaction_amount','sum'))
# step 4: calculate the variance of the daily transaction amounts
customer_var = df_customer_wt_missing_agg.groupby(by='customer_account_id', as_index=False).agg(daily_transaction_variance = ('total_transaction_amount','var'))
customer_var['daily_transaction_variance'] = customer_var['daily_transaction_variance'].astype('int64')


In [125]:
customer_var.head()

,customer_account_id,daily_transaction_variance
0,a_0,4127577
1,a_107,595651
2,a_109,227425
3,a_110,3130607
4,a_112,1506203


In [126]:
# data quality check 
# ensure these customer account id are without any missing value in balance
# previously we know there are 85 out of 150 customers with the missing value 
# the number of the rest of customer should be 150-85 = 65
customer_var.customer_account_id.nunique()

65

In [127]:
# step 5: return the customer_account_id that has the highest variance in daily transaction amount
customer_var.sort_values(by='daily_transaction_variance', ascending=False).head()

,customer_account_id,daily_transaction_variance
59,a_91,344160976
49,a_65,169687253
18,a_144,117732662
21,a_148,91673075
5,a_114,85188075


In [128]:
# validate the result via calculating variance using np.var() for the particular customer_account_id
a_91_transaction = df_customer_wt_missing_agg.loc[df_customer_wt_missing_agg.customer_account_id== 'a_91', 'total_transaction_amount'].tolist()
np.var(a_91_transaction)


340538228.9566759

In [28]:
print('{} has the largest variance of the sums of daily transactions amounts across all accounts?'.format('a_91'))

a_91 has the largest variance of the sums of daily transactions amounts across all accounts?


(b) Among the `customer_account_id`s without any missing or null/NaN beginning-of-day balance for any date in between their first and their last `accrual_date`s, which `customer_account_id` has the largest variance of the EoD (end-of-day, not beginning-of-day) global balances? ( 2 points )

In [29]:
# step1: calculate end-of-day balance 
# end-of-day balance (EoD) = beginning-of-day balance (BoD) + tota transaction amount 
df_customer_wt_missing_agg['balance_amount_EoD'] = df_customer_wt_missing_agg['balance_amount'] + df_customer_wt_missing_agg['total_transaction_amount']
df_customer_wt_missing_agg.head()

,customer_account_id,account_id,accrual_date,balance_amount,total_transaction_amount,balance_amount_EoD
0,a_0,a_0_0,2019-12-24,9950997.0,-248.0,9950749.0
1,a_0,a_0_0,2019-12-25,9950749.0,-859.0,9949890.0
2,a_0,a_0_0,2019-12-26,9949890.0,-776.0,9949114.0
3,a_0,a_0_0,2019-12-27,9949114.0,-605.0,9948509.0
4,a_0,a_0_0,2019-12-28,9948509.0,-33.0,9948476.0


In [109]:
# step2: roll it up to customer_account_id and accrual_date level to get the EoD balance per day
customer_balance_daily = df_customer_wt_missing_agg.groupby(by=['customer_account_id','accrual_date'], as_index=False).agg(daily_balance_amount_EoD = ('balance_amount_EoD','sum'))
# step3: calculate the variance of the EoD globel accounts per account id
customer_balance_var = customer_balance_daily.groupby(by=['customer_account_id'], as_index=False).agg(daily_balance_variance = ('daily_balance_amount_EoD','var'))


In [111]:
# step3: return the customer_account_id that has the highest variance  the EoD global balances
customer_balance_var['daily_balance_variance'] = customer_balance_var['daily_balance_variance'].astype('int64')
customer_balance_var.sort_values(by='daily_balance_variance', ascending=False).head()

,customer_account_id,daily_balance_variance
3,a_110,372505915518462
26,a_2,313327876573938
23,a_15,271221007024740
8,a_118,258427337767706
12,a_125,252537192404649


In [114]:
# validate the result using variance() 
test_list = customer_balance_daily.loc[customer_balance_daily.customer_account_id == 'a_110', 'daily_balance_amount_EoD'].tolist()
  
# Function will automatically calculate
# it's mean and set it as xbar
print("Variance of sample set is % s" %(statistics.variance(test_list)))

Variance of sample set is 372505915518462.3


### Question 4. 

Recommended time usage: ~60+ minutes

(a) Calculate the `end-of-day` global balance for `Customer B` on `Customer B Balance Date`. Fill in this EoD balance as the answer in the submission csv. ( 4 points )

In [131]:
# use customer_b and/or customer_b_balance_date
# (a)
# Based on the previous rule, we realize that end-of-day balance is calculated by the sum of the BoD balance and daily transactions across accounts
# However, we may notice that there are no balanced_id/ balance_amount associated with some of the accounts
df_transactions_balances[(df_transactions_balances.customer_account_id == customer_b) & (df_transactions_balances.accrual_date == customer_b_balance_date)]


,account_id,accrual_date,transaction_amount,transaction_id,customer_account_id,customer_name,account_name,balance_id,balance_amount
50869,a_32_0,2020-08-12,-661.0,a_32_0_155,a_32,a_32_company,a_32_0_fin_account,NaN,NaN
51025,a_32_1,2020-08-12,-301.0,a_32_1_135,a_32,a_32_company,a_32_1_fin_account,NaN,NaN


In [132]:
# step1: aggregate the transactions
# We need to fill missing values with 0 otherwise group by will drop out all the nulls in groupby
pd.set_option("display.max_rows", None, "display.max_columns", None) 
groupby_cols = ['customer_account_id', 'account_id', 'accrual_date','balance_amount']
customerB_agg = df_transactions_balances[df_transactions_balances.customer_account_id == customer_b].fillna(0).groupby(by=groupby_cols, as_index=False).agg(total_transaction_amount = ('transaction_amount','sum'))
customerB_agg.head(5)

,customer_account_id,account_id,accrual_date,balance_amount,total_transaction_amount
0,a_32,a_32_0,2020-03-10,9960127.0,-670.0
1,a_32,a_32_0,2020-03-11,9959457.0,333.0
2,a_32,a_32_0,2020-03-12,9959790.0,-1094.0
3,a_32,a_32_0,2020-03-13,9958696.0,-1147.0
4,a_32,a_32_0,2020-03-14,9957549.0,-385.0


In [133]:
# step2: identify the start date when missing balance event begins 
# It is apparent that the balance amount is 0 starting from 2020-07-30
# in this case, we have to borrow the balance_amount from the previous date which is 2020-07-29 
# and substract by previous total transaction amount to obtain today's beginning-of-day (BoD) balance
customerB_agg[customerB_agg.balance_amount==0].head()

,customer_account_id,account_id,accrual_date,balance_amount,total_transaction_amount
142,a_32,a_32_0,2020-07-30,0.0,-225.0
143,a_32,a_32_0,2020-07-31,0.0,-332.0
144,a_32,a_32_0,2020-08-01,0.0,-553.0
145,a_32,a_32_0,2020-08-02,0.0,-533.0
146,a_32,a_32_0,2020-08-03,0.0,-406.0


In [134]:
customerB_agg['balance_amount_BoD'] = customerB_agg['balance_amount']
customerB_agg['balance_amount_EoD'] = customerB_agg['balance_amount'] + customerB_agg['total_transaction_amount']

In [135]:
customerB_agg.head()

,customer_account_id,account_id,accrual_date,balance_amount,total_transaction_amount,balance_amount_BoD,balance_amount_EoD
0,a_32,a_32_0,2020-03-10,9960127.0,-670.0,9960127.0,9959457.0
1,a_32,a_32_0,2020-03-11,9959457.0,333.0,9959457.0,9959790.0
2,a_32,a_32_0,2020-03-12,9959790.0,-1094.0,9959790.0,9958696.0
3,a_32,a_32_0,2020-03-13,9958696.0,-1147.0,9958696.0,9957549.0
4,a_32,a_32_0,2020-03-14,9957549.0,-385.0,9957549.0,9957164.0


In [136]:
# loop over each row, if the account_id is the same as the previous record 
# then perform the tranformation
# 1) if the account_id in last row is not equal to the current row then we can't borrow the balance
# 2) calculate today's BoD by summing up yesterday's BoD and daily transaction 
# 3) update the EoD by adding the daily transactions to the BoD we got in previous step
for i in range(1,len(customerB_agg)):
    if customerB_agg.loc[i,'account_id'] == customerB_agg.loc[i-1,'account_id']: 
        customerB_agg.loc[i,'balance_amount_BoD'] = customerB_agg.loc[i-1, 'balance_amount_BoD'] + customerB_agg.loc[i-1, 'total_transaction_amount'] # calculate today's BoD by calculating the sum of 
        customerB_agg.loc[i,'balance_amount_EoD'] = customerB_agg.loc[i,'balance_amount_BoD'] + customerB_agg.loc[i,'total_transaction_amount'] # update the EoD in each loop
        

In [137]:
customerB_gl_balance = customerB_agg.loc[customerB_agg.accrual_date == customer_b_balance_date, 'balance_amount_EoD'].sum()

In [138]:
print('The end-of-day global balance for Customer B on Customer B Balance Date is {}'.format(customerB_gl_balance))

The end-of-day global balance for Customer B on Customer B Balance Date is 20019370.0


(b) Compute the `end-of-day` balances for all `financial_accounts`. Fill in as many missing dates as possible. The final dataset should include daily `end-of-day` balances for all `financial_accounts` as a csv in a similar format as `financial_balances.csv` (include just those columns: [`account_id, accrual_date, amount`]) and named `financial_balances_submission.csv`. ( 10 points )


In [142]:
# take a look at records within missing balance
df_transactions_balances[df_transactions_balances.balance_amount.isnull()].head()

,account_id,accrual_date,transaction_amount,transaction_id,customer_account_id,customer_name,account_name,balance_id,balance_amount
2909,a_1_0,2020-02-28,-691.0,a_1_0_0,a_1,a_1_company,a_1_0_fin_account,NaN,NaN
2910,a_1_0,2020-02-29,-820.0,a_1_0_1,a_1,a_1_company,a_1_0_fin_account,NaN,NaN
2911,a_1_0,2020-03-01,-624.0,a_1_0_2,a_1,a_1_company,a_1_0_fin_account,NaN,NaN
2912,a_1_0,2020-03-02,-972.0,a_1_0_3,a_1,a_1_company,a_1_0_fin_account,NaN,NaN
2913,a_1_0,2020-03-03,-510.0,a_1_0_4,a_1,a_1_company,a_1_0_fin_account,NaN,NaN


In [40]:
# step1: apply the same logic we concluded in last question without specifying customer_account_id
customer_agg_fillna = df_transactions_balances.fillna(0).groupby(by=groupby_cols, as_index=False).agg(total_transaction_amount = ('transaction_amount','sum'))

In [41]:
# step2: specify initial BoD and EoD
customer_agg_fillna['balance_amount_BoD'] = customer_agg_fillna['balance_amount']
customer_agg_fillna['balance_amount_EoD'] = customer_agg_fillna['balance_amount'] + customer_agg_fillna['total_transaction_amount']

In [42]:
# step3: loop through each rows and update the current balance_amount_BoD based on previous BoD and transactions
for i in range(1,len(customer_agg_fillna)):
    if customer_agg_fillna.loc[i,'account_id'] == customer_agg_fillna.loc[i-1,'account_id']: # if the account_id in last index not equal to the current row then we can't borrow the balance
        customer_agg_fillna.loc[i,'balance_amount_BoD'] = customer_agg_fillna.loc[i-1, 'balance_amount_BoD'] + customer_agg_fillna.loc[i-1, 'total_transaction_amount']
        customer_agg_fillna.loc[i,'balance_amount_EoD'] = customer_agg_fillna.loc[i,'balance_amount_BoD'] + customer_agg_fillna.loc[i,'total_transaction_amount'] # update the EoD in each loop
        

In [43]:
# step4: write to a csv file with selected columns
customer_agg_fillna = customer_agg_fillna.rename(columns={'balance_amount_EoD':'amount'})

In [44]:
customer_agg_fillna[['account_id', 'accrual_date', 'amount']].to_csv('financial_balances_submission.csv')